### Парсинг данных и создание датафрейма

In [ ]:
import os
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
paths = glob.glob(f"build/*ParticleData_t*.csv")

In [ ]:
# Вытаскиваем из первого файла названия столбцов
with open(paths[0], 'r') as f:
    lines = []
    while True:
        line = f.readline()
        if line[0] != '#':
            break
        lines.append(line)
    title = lines[1].replace('#title ', '')[:-1]                        # переделать через regex
    sep = chr(int(lines[2].replace('#separator ', '')[:-1]))            # переделать через regex
    vsc_sep = chr(int(lines[3].replace('#vector_separator ', '')[:-1])) # переделать через regex
    columns = [' '.join(i.split(' ')[2:])[:-1] for i in lines[4:]]
    metarows_len = len(lines)

In [ ]:
data = pd.DataFrame()
for path in paths:
    item = pd.read_csv(path, skiprows=metarows_len, names=columns)
    data = data.append(item, ignore_index=True)

In [ ]:
data

In [ ]:
set(data["particle_name"])

In [ ]:
len(data[data["particle_name"]=="e-"])

In [ ]:
len(data[data["particle_name"]=="gamma"])

### Преобразование к полярным координатам

In [ ]:
xyz_e = data[data["particle_name"]=="gamma"][:1000]
xdata = xyz_e["X"].to_numpy()
ydata = xyz_e["Y"].to_numpy()
zdata = xyz_e["Z"].to_numpy()
xyz = np.array([xdata, ydata, zdata]).T

In [ ]:
# Я это писал 6 часов, поскольку не думал что формулы из вики или матанализа не работают
def cart2sph(n_xyz: np.array, trh=10**-9):
    ra, θa, φa = [], [], []
    for x, y, z in n_xyz:
        r = np.sqrt(x**2 + y**2 + z**2)
        θ = 0. if np.abs(r) < trh else np.arccos(z/r)
        if x>0 and y>0:
            φ = np.arctan(y/x)
        elif np.abs(x) < trh and y>0:
            φ = np.pi/2
        elif x<0 and y>0:
            φ = np.pi + np.arctan(y/x)
        elif x<0 and y<0:
            φ = np.pi + np.arctan(y/x)
        elif np.abs(x) < trh and y<0:
            φ = np.pi*3/4
        elif x>0 and y<0:
            φ = 2*np.pi + np.arctan(y/x)
        elif np.abs(x) < trh and np.abs(y) < trh:
            φ = 0.
        ra.append(r)
        θa.append(θ)
        φa.append(φ)
    return np.array([ra, θa, φa]).T

def sph2cart(n_rθφ: np.array):
    n_rθφ[:, 2] = n_rθφ[:, 2] % (2*np.pi)
    n_rθφ[:, 1] = n_rθφ[:, 1] % (np.pi)
    x = n_rθφ[:, 0] * np.sin(n_rθφ[:, 1]) * np.cos(n_rθφ[:, 2])
    y = n_rθφ[:, 0] * np.sin(n_rθφ[:, 1]) * np.sin(n_rθφ[:, 2])
    z = n_rθφ[:, 0] * np.cos(n_rθφ[:, 1])
    return np.array([x, y, z]).T

In [ ]:
nrθφ = cart2sph(xyz)

In [ ]:
print(f"θ range: {min(nrθφ[:,1])} to {max(nrθφ[:,1])}")
print(f"φ range: {min(nrθφ[:,2])} to {max(nrθφ[:,2])}")

### Выбор интересующей области

In [ ]:
θ_from = np.pi/2-np.pi/10
θ_to = np.pi/2+np.pi/10.

In [ ]:
segment, out_segment = [], []
i = 0
j = 0
for r, theta, phi in nrθφ:
    if θ_from<theta and theta<θ_to:
        segment.append((1, theta, phi))
        i += 1
    else:
        out_segment.append((1, theta, phi))
        j += 1
    
xyz_segment = sph2cart(np.array(segment))
xyz_out_segment = sph2cart(np.array(out_segment))

In [ ]:
fig = plt.figure(figsize = (6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*xyz_segment.T, c="red")
ax.scatter(*xyz_out_segment.T, c="blue")
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(-1, 1)
plt.show()

In [ ]:
segment_theta = np.array(segment).T[1] # Select phi
hist = np.histogram(segment_theta, bins=100)
segment_theta.shape

In [ ]:
theta = hist[1][:100]
radii = hist[0]

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, radii)
ax.set_title("Диаграмма направленности рентгеновского излучения.", va='bottom')
plt.show()